# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except:
    print("Error while creating the cluster")

#### Create Keyspace

In [6]:
#Creation of a Keyspace
session.execute("""
create keyspace if not exists project
with replication={'class' : 'SimpleStrategy', 'replication_factor' : 1}
""")

#### Set Keyspace

In [7]:
#Setting KEYSPACE to the keyspace specified above
session.set_keyspace('project')

### Create queries to ask the following three questions of the data
#### 1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
#### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

## Query #1

## Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

#### To answer this question we will need to get (select) the artist name, song name, and the song length from our table, and also we will need to filter the results by sessionId and itemInSession.

#### Steps to be followed are:
##### 1. Create a table by the name artist_session if it doesn't exist by using If exists clause.
##### 2. Insert values into the created table by reading the event datafile.
##### 3. Writing the select query in CQL to fetch the required results. The select query is mentioned just below this statment.
##### select artist, song, length from artist_session where sessionId= 338 and itemInSession= 4

#### As mentioned in our point 1, We will name our table artist_session
#### Our primary key will consist of partition key sessionId, and clustering key itemInSession so as to filter by this later on as mentioned in the select statement
#### The columns of our table are: sessionId, itemInSession, artist, length (song) and song (name).

In [8]:
session.execute("""
create table if not exists artist_session(sessionId int, itemInSession int, artist text, length float, song text, primary key(sessionId, itemInSession))
""")            

In [9]:
# Loading the table with the necessary parameter mentioned in the above step from event_datafile_new.csv 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        #print (line[0])
        query = "insert into artist_session(sessionId, itemInSession, artist, length, song)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], float(line[5]), line[9]))

#### Selecting to verify that the data have been inserted into artist_session table

In [10]:
rows= session.execute("""
select artist, song, length from artist_session
where sessionId= 338 and itemInSession= 4
""")
for row in rows:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


## Query #2

## Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

#### To answer this question we will need to get (select) the itemInSession, artist name, song name, User name (First and last) from our table, and also we will need to filter the results by Userid, and sessionId.

#### Steps to be followed are:
##### 1. Create a table by the name user_session if it doesn't exist by using If exists clause
##### 2. Insert values into the created table by reading the event datafile.
##### 3. Writing the select query in CQL to fetch the required results. The select query is mentioned just below this statment.
##### select itemInSession, artist, song, firstName, lastName from user_session where userId= 10 and sessionId= 182

#### As mentioned in our point 1, We will name our table user_session
#### Our primary key contains userId, sessionId as partition keys and itemInSession as the clustering key so as to filter by this later on as mentioned in the select statement
#### The columns of our table are: itemInSession, artist, song, firstName, lastName.

In [11]:
session.execute("""
create table if not exists user_session(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, \
lastName text, primary key((userId, sessionId), itemInSession))
""") 

In [12]:
# Loading the table with the necessary parameter mentioned in the above step from event_datafile_new.csv
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into user_session(userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        

#### Selecting to verify that the data have been inserted into user_session table

In [13]:
rows= session.execute("""
select itemInSession, artist, song, firstName, lastName from user_session
where userId= 10 and sessionId= 182
""")
#print(rows)
for row in rows:
    #print(row)
    print(row.iteminsession, row.artist, row.song, row.firstname, row.lastname)

0 Down To The Bone Keep On Keepin' On Sylvie Cruz
1 Three Drives Greece 2000 Sylvie Cruz
2 Sebastien Tellier Kilometer Sylvie Cruz
3 Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## Query #3

## Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### To answer this question we will need to get (select) the User name (First and last) from our table, and also we will need to filter the results by Song Name.

#### Steps to be followed are:
##### 1. Create a table by the name music_session if it doesn't exist by using If exists clause
##### 2. Insert values into the created table by reading the event datafile.
##### 3. Writing the select query in CQL to fetch the required results. The select query is mentioned just below this statment.
##### select firstName, lastName from music_session where song= 'All Hands Against His Own'

#### As mentioned in our point 1, We will name our table music_session
#### Our primary key contains song as partition keys and userID as the clustering key so as to filter by this later on as mentioned in the select statement
#### The columns of our table are: firstName, lastName.

In [14]:
session.execute("""
create table if not exists music_session(userId int, firstName text, lastName text, song text, primary key(song, userId))
""") 
                    

In [15]:
# Loading the table with the necessary parameter mentioned in the above step from event_datafile_new.csv
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into music_session(userId, firstName, lastName, song)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (int(line[10]), line[1], line[4], line[9]))

#### Selecting to verify that the data have been inserted into music_session table

In [16]:
rows= session.execute("""
select firstName, lastName from music_session
where song= 'All Hands Against His Own'
""")
#print(rows)
for row in rows:
    #print(row)
    print(row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Dropping the tables before closing out the sessions

In [ ]:
session.execute("drop table if exists artist_session")
session.execute("drop table if exists user_session")
session.execute("drop table if exists music_session")

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()